In [1]:
!pip install -q accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.7 MB/s eta 0:00:00


In [ ]:
# import torch
# import os
# from google.colab import userdata
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# from torch.quantization import quantize_dynamic

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# access_token = userdata.get("HF_TOKEN")

# quantization_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     llm_int8_threshold=6.0,
#     llm_int8_skip_modules=None,
#     llm_int8_enable_fp32_cpu_offload=False,
# )

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tokenizer = AutoTokenizer.from_pretrained(model_id, token=access_token)
# tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=quantization_config,
#     device_map="auto",
#     token=access_token,
# )

# system_prompt = "Você é um assistente prestativo que responde em português brasileiro."

# user_input = "Qual é a capital do Brasil?"

# messages = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": user_input},
# ]

# chat_template = tokenizer.chat_template or ""
# input_ids = tokenizer.apply_chat_template(
#     messages,
#     tokenize=True,
#     add_generation_prompt=True,
#     return_tensors="pt",
# ).to(device)

# with torch.no_grad():
#     output_ids = model.generate(
#         input_ids,
#         max_length=512,
#         temperature=0.7,
#         top_p=0.9,
#         do_sample=True,
#         eos_token_id=tokenizer.eos_token_id,
#         pad_token_id=tokenizer.pad_token_id,
#     )

# response = tokenizer.decode(output_ids[0][input_ids.shape[-1]:],
#                             skip_special_tokens=True)

# print("Pergunta:", user_input)
# print("\nResposta do modelo:")
# print(response)

# print("\nTamanho do modelo em MB:", model.get_memory_footprint() / (1024 * 1024))
# print("Tipo de dispositivo:", device)

In [ ]:
# def predict(user_input):
#     system_prompt = "Você é um assistente prestativo que responde em português brasileiro."

#     messages = [
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": user_input},
#     ]

#     input_ids = tokenizer.apply_chat_template(
#         messages,
#         tokenize=True,
#         add_generation_prompt=True,
#         return_tensors="pt",
#     ).to(device)

#     with torch.no_grad():
#         output_ids = model.generate(
#             input_ids,
#             max_length=512,
#             temperature=0.7,
#             top_p=0.9,
#             do_sample=True,
#             eos_token_id=tokenizer.eos_token_id,
#             pad_token_id=tokenizer.pad_token_id,
#         )

#     response = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

#     return response

# result = predict("Qual é a capital do Brasil?")
# print("Resposta:", result)

In [9]:
import os
from google.colab import userdata
import torch
import logging
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextIteratorStreamer
from threading import Thread

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class LlamaQuantizedModel:
    def __init__(self, model_id="meta-llama/Meta-Llama-3-8B-Instruct", system_prompt=None):
        self.model_id = model_id
        self.logger = logging.getLogger(__name__)

        self.logger.info(f"Inicializando LlamaQuantizedModel com modelo: {model_id}")

        self.access_token = os.getenv("HF_TOKEN")

        if not self.access_token:
            self.logger.error("HF_TOKEN não encontrado nas variáveis de ambiente")
            raise ValueError("HF_TOKEN não encontrado nas variáveis de ambiente")

        self.logger.info("Token do Hugging Face carregado com sucesso")

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.logger.info(f"Dispositivo utilizado: {self.device}")

        self.system_prompt = system_prompt or "Você é um assistente prestativo que responde em português brasileiro."

        self.tokenizer = None
        self.model = None

        self._load_model()

    def _load_model(self):
        print(f"Carregando tokenizer para {self.model_id}...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id, token=self.access_token)
        self.tokenizer.pad_token = self.tokenizer.eos_token

        print(f"Carregando modelo quantizado para {self.model_id}...")
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
        )

        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            quantization_config=quantization_config,
            device_map="auto",
            token=self.access_token
        )

        print("Modelo carregado com sucesso!")

    def predict(self, user_input, max_length=512, temperature=0.7, top_p=0.9):
        self.logger.info(f"Iniciando predição com input: {user_input[:50]}...")
        self.logger.debug(f"Parâmetros - max_length: {max_length}, temperature: {temperature}, top_p: {top_p}")

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_input},
        ]

        self.logger.debug("Aplicando chat template e tokenizando...")
        input_ids = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to(self.device)

        self.logger.info(f"Shape dos tokens de entrada: {input_ids.shape}")

        self.logger.debug("Gerando output do modelo...")
        with torch.no_grad():
            output_ids = self.model.generate(
                input_ids,
                max_length=max_length,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.pad_token_id,
            )

        self.logger.debug("Decodificando resposta...")
        response = self.tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

        self.logger.info(f"Predição completada. Resposta com {len(response)} caracteres")

        return response

    def set_system_prompt(self, new_prompt):
        self.logger.info(f"Alterando system prompt")
        self.system_prompt = new_prompt
        self.logger.debug(f"Novo system prompt: {new_prompt}")

    def get_device_info(self):
        self.logger.info("Coletando informações do dispositivo")
        info = {
            "device": str(self.device),
            "memory_allocated_gb": torch.cuda.memory_allocated() / (1024**3) if torch.cuda.is_available() else 0,
        }
        self.logger.debug(f"Informações do dispositivo: {info}")
        return info

    def predict_stream(self, user_input, max_length=512, temperature=0.7, top_p=0.9):
        self.logger.info(f"Iniciando predição com streaming para input: {user_input[:50]}...")
        self.logger.debug(f"Parâmetros - max_length: {max_length}, temperature: {temperature}, top_p: {top_p}")

        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_input},
        ]

        self.logger.debug("Aplicando chat template e tokenizando...")
        input_ids = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to(self.device)

        self.logger.info(f"Shape dos tokens de entrada: {input_ids.shape}")

        streamer = TextIteratorStreamer(
            self.tokenizer,
            skip_prompt=True,
            skip_special_tokens=True,
        )

        self.logger.debug("Iniciando geração com streaming em thread separada...")
        generation_kwargs = {
            "input_ids": input_ids,
            "streamer": streamer,
            "max_length": max_length,
            "temperature": temperature,
            "top_p": top_p,
            "do_sample": True,
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id,
        }

        thread = Thread(target=self.model.generate, kwargs=generation_kwargs)
        thread.start()

        self.logger.info("Transmitindo tokens em tempo real...")
        return streamer

    def unload_model(self):
        self.logger.info("Iniciando descarregamento do modelo")
        if self.model is not None:
            del self.model
            del self.tokenizer
            torch.cuda.empty_cache()
            self.logger.info("Modelo descarregado com sucesso da memória")


In [10]:
llama_model = LlamaQuantizedModel()

result = llama_model.predict("Qual é a capital do Brasil?")
print("Resposta:", result)

device_info = llama_model.get_device_info()
print("\nInformações do dispositivo:", device_info)

Carregando tokenizer para meta-llama/Meta-Llama-3-8B-Instruct...
Carregando modelo quantizado para meta-llama/Meta-Llama-3-8B-Instruct...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Modelo carregado com sucesso!
Resposta: A capital do Brasil é Brasília!

Informações do dispositivo: {'device': 'cuda', 'memory_allocated_gb': 10.636491775512695}


In [5]:
%%time
result = llama_model.predict("Qual é a capital do Brasil?")
print("Resposta:", result)

Resposta: A capital do Brasil é Brasília!
CPU times: user 1.13 s, sys: 0 ns, total: 1.13 s
Wall time: 3.06 s


In [6]:
%%time
result = llama_model.predict("o que você faz?")
print("Resposta:", result)

Resposta: Eu sou um assistente prestativo treinado para ajudar com informações e tarefas em português brasileiro. Eu posso:

* Responder perguntas sobre um assunto específico ou geral
* Fornecer informações sobre um tema ou tópico
* Ajuda a encontrar recursos online ou offline
* Traduzir textos simples de uma língua para outra (não é uma tradução profissional, mas pode ajudar em casos simples)
* Geração de textos, como emails, cartas ou resumos
* Simular uma conversa, como se fosse um bate-papo
* Fornecer sugestões e ideias para resolver problemas ou melhorar situações

Se você tiver alguma dúvida ou necessidade específica, sinta-se à vontade para perguntar!
CPU times: user 20.8 s, sys: 0 ns, total: 20.8 s
Wall time: 37.3 s


In [11]:
%%time
streamer = llama_model.predict_stream("Me conte uma história interessante em 100 palavras")

print("Resposta com streaming: ", end="", flush=True)
for text in streamer:
    print(text, end="", flush=True)
print("\n")

Resposta com streaming: Que delícia! Aqui vai uma história interessante para você:

Em 1943, um homem chamado Victor Frankl foi enviado para um campo de concentração nazista. Ele era um psiquiatra e um escritor, e sua experiência lá inspirou seu livro mais famoso, "Man's Search for Meaning". Durante sua internação, Victor descobriu que, apesar da perda de tudo que ele conhecera, havia algo que o mantinha vivo: a busca por significado. Ele começou a dar aulas para os outros prisioneiros sobre a importância da busca por significado, e isso os ajudou a sobreviver àqueles tempos difíceis.

CPU times: user 13.3 s, sys: 76.5 ms, total: 13.4 s
Wall time: 13.6 s
